In [1]:
%matplotlib inline

import os
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


def get_frames_from_events(xaddr_all, yaddr_all, num_events_per_sample=2000,
                           chip_size=(240, 180), target_size=(64, 64)):
    """
    Extract ``num_events_per_sample`` events from a one-dimensional sequence of
    AER-events. The events are spatially subsampled to ``target_size``, and
    standardized to [0, 1] using 3-sigma normalization. The resulting events are binned 
    into a frame. The function operates on the events in ``xaddr_all`` etc 
    sequentially until all are processed into frames.
    """

    num_samples = int(len(xaddr_all) / num_events_per_sample)
    frames = np.zeros([num_samples] + list(target_size), 'float32')
    print("Extracting {} frames from DVS event sequence.".format(num_samples))
    # Iterate for as long as there are events in the sequence.
    for sample_idx in range(num_samples):
        event_idxs = range(num_events_per_sample * sample_idx,
                           num_events_per_sample * (sample_idx + 1))
        # Loop over ``num_events_per_sample`` events
        for x, y in zip(xaddr_all[event_idxs], yaddr_all[event_idxs]):
            # Subsample from 240x180 to e.g. 64x64
            x = int(x * (target_size[0] - 1) / (chip_size[0] - 1))
            y = int(y * (target_size[1] - 1) / (chip_size[1] - 1))
            # Count event at subsampled location (x and y axes are swapped)
            frames[sample_idx, y, x] += 1
        
        # Compute standard deviation of event-sum distribution 
        # after removing zeros
        sample = frames[sample_idx]
        sigma = np.std(sample[np.nonzero(sample)])
        
        # Clip number of events per pixel to three-sigma
        frames[sample_idx] = np.clip(sample, 0, 3*sigma)
        
    return frames / 255.

In [ ]:
datapath = '/home/rbodo/Downloads'
filename = 'rec_extracted.npz'

# Load recorded data. Assumes that the data has been extracted from the hdf5 
# file into variables like ``xaddr``, ``yaddr``, ``timestamp`` etc, and saved 
# as compressed numpy file, using a command like
# np.savez_compressed(os.path.join(datapath, filename), xaddr=xaddr, yaddr=yaddr, ...)
data_dict = np.load(os.path.join(datapath, filename))

print("Keys to data fields stored in ``data_dict``: {}".format(data_dict.keys()))

# Access individual fields by the name of the keyword argument 
# with which it was saved.
xaddr = data_dict['xaddr']
yaddr = data_dict['yaddr']

# The following are not needed for creating the frames.
timestamp = data_dict['timestamp']
sonar_left = data_dict['sonar_left']
sonar_center = data_dict['sonar_center']
sonar_right = data_dict['sonar_right']
throttle = data_dict['throttle']
steering = data_dict['steering']

In [ ]:
frames = get_frames_from_events(xaddr, yaddr, num_events_per_sample=2000,
                                chip_size=(240, 180), target_size=(64, 64))

In [ ]:
plt.imshow(frames[0])

TODO: May need to add an extra axis to ``frames`` to fit input shape of CNN, i.e. (1, 64, 64) instead of (64, 64). 